Universidade Federal do Rio Grande  
Instituto de Oceanografia  
Programa de Pós-graduação em Oceanologia  
**Disciplina**: Análises de Séries Temporais em Oceanografia – 2023  
**Estudante**: Andrés Eloy Piñango Jauregui (153423)  

# Lista de exercícios 3: Estatística Básica de Séries Temporais: Autocorrelação, Autocovariância, Integral de Escala de Tempo & Correlação Cruzada
****
**Atividades**:  

### Parte I – Estatística Básica
#### 1. Visualização dos dados (aconselha-se utilizar arquivo sea_level_ssh_tsl.mat)  
**a) Utilize a rotina imagesc.m do Matlab para visualizar os dados de ssh e tsl. Use subplots para apresentar ambos os dados comparativamente em um determinado tempo**


In [ ]:
### ------------------------------------------------------------------------------------
### Load the libraries to be used in the analysis
### ------------------------------------------------------------------------------------
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy import io
from scipy import stats

In [ ]:
### ------------------------------------------------------------------------------------
### Load the mat file
### ------------------------------------------------------------------------------------
data = io.loadmat("./Data/sea_level_ssh_tsl.mat")

In [ ]:
### ------------------------------------------------------------------------------------
### Transform the time values to actual time data
### ------------------------------------------------------------------------------------
a = np.datetime64("0000-01-01")  # Epoch used by matlab
b = np.datetime64("1970-01-01")  # Unix epoch, default in python
days = b - a
dates = data["time"].flatten() - days.astype(int)
time = pd.to_datetime(dates, unit="D")

In [ ]:
### ------------------------------------------------------------------------------------
### Create a DataArray for each variable from the mat file
### ------------------------------------------------------------------------------------
# Sea Surface Height (SSH) anomaly
ssh = xr.DataArray(
    data["ssh_anom"],
    dims=["lat", "lon", "time"],
    coords={
        "lat": data["lat_tsl"].flatten(),
        "lon": data["lon_tsl"].flatten(),
        "time": time,
    },
    attrs={
        "standard_name": "SSH anomaly",
        "long_name": "Sea Surface Height anomaly",
        "units": "m",
    },
)
# Thermosteric Sea Level (TSL) anomaly
tsl = xr.DataArray(
    data["tsl_anom"],
    dims=["lat", "lon", "time"],
    coords={
        "lat": data["lat_tsl"].flatten(),
        "lon": data["lon_tsl"].flatten(),
        "time": time,
    },
    attrs={
        "standard_name": "TSL anomaly",
        "long_name": "Thermosteric Sea Level anomaly",
        "units": "m",
    },
)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the data
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(14, 5), dpi=300)
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
### Subplot 1: Temperature
ssh.sel(time="2010-11-16").plot(
    ax=ax1, vmin=-0.07, vmax=0.07, cmap="RdBu_r", extend="both"
)
ax1.set_ylabel("Latitude")
ax1.set_xlabel("Longitude")
### Subplot 2: Partial pressure of carbon dioxide in seawater
tsl.sel(time="2010-11-16").plot(
    ax=ax2, vmin=-0.07, vmax=0.07, cmap="RdBu_r", extend="both"
)
ax2.set_ylabel("Latitude")
ax2.set_xlabel("Longitude")
### Final touch
fig.tight_layout(pad=0.2)
plt.show()

#### 2. Cálculo da Autocorrelação e Integral da Escala de Tempo
**a) Escolha uma determinada posição (lon, lat) para as análises**


In [ ]:
### ------------------------------------------------------------------------------------
### Define the coords for the analysis
### ------------------------------------------------------------------------------------
longitude = -55.5
latitude = 31.5

**b) Calcule a autocorrelação e a integral de escala de tempo para esse ponto usando a função time_scale.m fornecida**


In [ ]:
### ------------------------------------------------------------------------------------
### Define the function that calculate the autocorrelation
### ------------------------------------------------------------------------------------
def time_scale(series):
    # Calculate the autocorrelation of the series
    max_lag = int(np.ceil(len(series) / 2))
    series_cov = (
        series - series.mean()
    )  # In matlab the xcov function computes the mean of its inputs, subtracts the mean, and then calls xcorr.
    corr = np.correlate(series_cov, series_cov, mode="full")
    corr_cut = corr[
        (len(corr) // 2) - max_lag : (len(corr) // 2) + 1 + max_lag
    ]  # Imitate the job of the variable maxlag in the matlab function
    corr_sym = pd.Series(
        corr_cut[len(corr_cut) // 2 :]
    )  # keep only one side of the symmetric autocovariance
    corr_unb = (
        corr_sym * 1 / (len(series) - corr_sym.index.values)
    )  # From raw to unbiased cross-covariance.
    corr_norm = corr_unb / corr_unb[0]
    # Calculate the integral time scale
    time_scale = 0
    i = 0
    while corr_norm[i] >= 0:
        time_scale = time_scale + (corr_norm[i] + corr_norm[i + 1]) / 2
        i += 1
    return corr_norm, time_scale

In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the autocorrelation
### ------------------------------------------------------------------------------------
# SSH
ssh_point = ssh.sel(lon=longitude, lat=latitude).to_pandas()
ssh_corr, ssh_time = time_scale(ssh_point)
# TSL
tsl_point = tsl.sel(lon=longitude, lat=latitude).to_pandas()
tsl_corr, tsl_time = time_scale(tsl_point)

**c) Plot a autocorrelação para as duas séries ssh_anom e tsl_anom**


In [ ]:
### ------------------------------------------------------------------------------------
### Plot the autocorrelation
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
### Subplot 1: Temperature
ssh_corr.plot(ax=ax1)
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("SSH autocorrelation", loc="left")
ax1.set_ylabel("Autocorrelation Coefficient")
ax1.set_xlabel("Lag Time (months)")
### Subplot 2: TSL
tsl_corr.plot(ax=ax2)
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("TSL autocorrelation", loc="left")
ax2.set_ylabel("Autocorrelation Coefficient")
ax2.set_xlabel("Lag Time (months)")
### Final touch
fig.tight_layout(pad=0.2)
plt.show()

**d) Quais são as integrais de escala de tempo das duas séries para o ponto escolhido?**


In [ ]:
### ------------------------------------------------------------------------------------
### Print the info
### ------------------------------------------------------------------------------------
print(
    f"A integral da escala de tempo para os dados de SSH na coordenada 31.5°N, 55.5°W foi {round(ssh_time, 4)}"
)
print(
    f"A integral da escala de tempo para os dados de TSL na coordenada 31.5°N, 55.5°W foi {round(tsl_time, 4)}"
)

**e) Quantos são os graus de liberdade de cada série? Lembre-se que `N*=N/T*`**


In [ ]:
### ------------------------------------------------------------------------------------
### Print the info
### ------------------------------------------------------------------------------------
print(
    f"O número de graus de liberdade para os dados de SSH na coordenada 31.5°N, 55.5°W foi {round(len(ssh_point)/ssh_time, 4)}"
)
print(
    f"O número de graus de liberdade para os dados de TSL na coordenada 31.5°N, 55.5°W foi {round(len(tsl_point)/tsl_time, 4)}"
)

### 3. Cálculo da Correlação Cruzada
**a) Calcule a correlação cruzada entre as séries para o ponto escolhido com nível de 95% de confiança (correlate.m)**


In [ ]:
### ------------------------------------------------------------------------------------
### Print the info
### ------------------------------------------------------------------------------------
print(
    f"A correlação cruzada entre as séries SSH e TSL na coordenada 31.5°N, 55.5°W foi {round(ssh_point.corr(tsl_point), 4)}"
)

**b) Tente calcular a correlação cruzada para todos os pontos de sua grade espacial. Use loops de programação**


In [ ]:
### ------------------------------------------------------------------------------------
### Define the function that calculate the correlation and the p-value
### ------------------------------------------------------------------------------------
def correlate(da_a, da_b, dim):
    # Calculate the Pearson correlation
    corr = xr.corr(da_a, da_b, dim=dim)
    corr.attrs["long_name"] = "Correlation Coefficient"
    # Calculate the p-value
    n = len(da_a[dim])
    dist = stats.beta(n / 2 - 1, n / 2 - 1, loc=-1, scale=2)
    func = lambda x: 2 * dist.cdf(-abs(x))
    p_values = xr.apply_ufunc(func, corr)
    p_values.attrs["long_name"] = "p-value"
    return corr, p_values

In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the correlation and the p-value
### ------------------------------------------------------------------------------------
ssh_tsl, pvalues = correlate(ssh, tsl, "time")

**c) Mostre graficamente o coeficiente de correlação cruzada entre os dados de ssh_anom e tsl_anom. (Semelhante ao slide 25 da aula 4. Utilize imagesc.m, colorbar.m, etc)**  
  
**d) Mostre num gride espacial (imagesc.m) onde os coeficientes são estatisticamente significantes. (Utilize o exemplo do slide 25 da Aula4)**

In [ ]:
### ------------------------------------------------------------------------------------
### Plot both, the correlation and the significant results
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(5, 7), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
### Subplot 1: Correlation
ssh_tsl.plot(ax=ax1)
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_title("SSH vs TSL Correlation", loc="left")
ax1.set_ylabel("Latitude")
ax1.set_xlabel("Longitude")
### Subplot 2: Significance
pvalues.plot(ax=ax2, vmin=0, vmax=0.25, cmap="tab20b")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_title("Correlation p-values", loc="left")
ax2.set_ylabel("Latitude")
ax2.set_xlabel("Longitude")
### Final touch
fig.tight_layout(pad=0.2)
plt.show()

**e) Comente os resultados**  

Espacialmente, TSL e SST mostraram uma boa correlação (>0.7) entre 25 e 29°N e 54 e 59°W e uma baixa correlação no resto da região estudada. Em geral, todas as correlações são estatisticamente significativas (com 95%) exceto num pixel no extremo norte entre 58 e 57°W, onde, além disso, a correlação foi muito baixa (~0.1).

****
### Parte II – Periodicidade
Nessa sessão, iremos analisar as tendências e periodicidade das séries temporais utilizando estatística básica. Duas séries temporais serão analisadas neste exercício:
* O índice da oscilação sul, obtida de: https://www.ncdc.noaa.gov/teleconnections/enso/indicators/soi/#soi-calculation
* A concentração de CO2 na atmosfera na ilha de Mauna Loa, obtida de: ftp://aftp.cmdl.noaa.gov/products/trends/co2/co2_mm_mlo.txt

#### 4. Use a série de concentração de CO2 de Mauna Loa para as atividades a seguir
**a) Leia a série de dados. Verifique se existem períodos sem dados. Se existirem, tente preencher as lacunas com a função myinterp.m**


In [ ]:
### ------------------------------------------------------------------------------------
### Load the data and interpolate the gaps
### ------------------------------------------------------------------------------------
pco2_mau = pd.read_table(
    "./Data/co2_monthly_mauna_loa.txt",
    names=["year", "month", "decimal_date", "pco2", "interpolated", "trend", "days"],
    skiprows=72,
    sep="\s+",
    na_values=-99.99,
)
pco2_mau["time"] = pd.to_datetime(
    {"year": pco2_mau["year"], "month": pco2_mau["month"], "day": 15}
)
pco2_mau = (
    pco2_mau.drop(
        columns=["year", "month", "decimal_date", "interpolated", "trend", "days"]
    )
    .set_index("time")
    .squeeze()
)
pco2_mau2 = pco2_mau.interpolate()

**b) Plote a série temporal interpolada sobreposta à serie original. Existe alguma tendência na série?**


In [ ]:
### ------------------------------------------------------------------------------------
### Plot the figure
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 4), dpi=300)
ax = fig.add_subplot(111)
ax.grid(linestyle="dashed", alpha=0.3)
ax.plot(pco2_mau2, color="blue", label="Interpolated")
ax.plot(pco2_mau, color="#E71D36", label="Original")
ax.legend()
ax.set_title("Monthly mean CO2 mole fraction in Mauna Loa", loc="left")
ax.set_ylabel("Molar fraction")
### Final touch
fig.tight_layout(pad=0.2)
plt.show()

Sim, existe uma tendência na série. Como resultado da utilização de combustíveis fósseis, mudanças no uso da terra e outras atividades antrópicas, a concentração de CO2 na atmosfera está incrementando a uma taxa de ~2 μatm por ano.

**c) Remova a tendência utilizando a função detrend.m. Perceba que essa função remove a tendência linear da série. Plote novamente a série que você retirou a tendência linear. Você acha que a tendência foi removida corretamente?**

In [ ]:
### ------------------------------------------------------------------------------------
### Define a function to detrend the data
### ------------------------------------------------------------------------------------
def linear_detrend(series):
    new_series = series.reset_index(drop=True)
    clean_series = new_series.dropna()
    regression = np.polynomial.Polynomial.fit(
        clean_series.index.values, clean_series.values, deg=1
    )
    coef = regression.convert().coef
    detrended_series = series - ((coef[1] * new_series.index.values) + coef[0])
    return detrended_series

In [ ]:
### ------------------------------------------------------------------------------------
### Detrend the data
### ------------------------------------------------------------------------------------
pco2_lin_detre = linear_detrend(pco2_mau2)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the detrended data
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 4), dpi=300)
ax = fig.add_subplot(111)
ax.grid(linestyle="dashed", alpha=0.3)
ax.plot(pco2_lin_detre, color="#E71D36", label="Original")
ax.set_title(
    "Detrended monthly mean CO2 mole fraction in Mauna Loa\n1 degree polynom",
    loc="left",
)
ax.set_ylabel("Molar fraction")
### Final touch
fig.tight_layout(pad=0.2)
plt.show()

A função não removeu corretamente a tendência da série. Como o incremento do CO2 na atmosfera não é linear, outra função (polinômica, exponencial) poderia dar um melhor resultado.

**d) Tente ajustar agora uma função de 3° grau aos dados. Utilize as funções polyfit.m e polyval.m do Matlab**


In [ ]:
### ------------------------------------------------------------------------------------
### Define a second function to detrend the data
### ------------------------------------------------------------------------------------
def cubic_detrend(series):
    new_series = series.reset_index(drop=True)
    clean_series = new_series.dropna()
    regression = np.polynomial.Polynomial.fit(
        clean_series.index.values, clean_series.values, deg=3
    )
    coef = regression.convert().coef
    detrended_series = series - (
        (coef[3] * new_series.index.values**3)
        + (coef[2] * new_series.index.values**2)
        + (coef[1] * new_series.index.values)
        + coef[0]
    )
    return detrended_series

**e) Repita novamente o item (c)**


In [ ]:
### ------------------------------------------------------------------------------------
### Detrend the data
### ------------------------------------------------------------------------------------
pco2_cub_detre = cubic_detrend(pco2_mau2)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the detrended data
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 4), dpi=300)
ax = fig.add_subplot(111)
ax.grid(linestyle="dashed", alpha=0.3)
ax.plot(pco2_cub_detre, color="#E71D36", label="Original")
ax.set_title(
    "Detrended monthly mean CO2 mole fraction in Mauna Loa\n3 degree polynom",
    loc="left",
)
ax.set_ylabel("Molar fraction")
### Final touch
fig.tight_layout(pad=0.2)
plt.show()

**f) Comente os resultados**

Aplicando uma função polinômica de ordem 3, a tendência dos dados foi eliminada de forma satisfatória. Agora, os valores encontram-se simetricamente ao redor do zero tanto no início como no fim da série.

#### 5. Use o script abaixo para aprender a utilizar as funções xcov ou xcorr
````
dt = 0.5; T = 10; N = 200; T = [0:N-1].*dt;
Y = 2.*cos((2.*pi./T).*t);
[c,lags] = xcorr(y,'coeff');
figure; 
subplot(211); plot(t,y);
subplot(212); plot(lags,c)
````
Observe que uma série temporal de frequência 1/T (= 0.1 s) foi gerada a partir da função periódica cosseno. A série possui 200 valores.
* Qual é a frequência de amostragem da série?
* O que acontece com a função autocorrelação (xcorr.m) se diminuirmos ou aumentarmos N? Faça o teste.
* Plote e comente os resultados.


In [ ]:
### ------------------------------------------------------------------------------------
### Define a function for autoccorrelation
### ------------------------------------------------------------------------------------
def xcorr(serie_a, serie_b):
    corr = np.correlate(serie_a, serie_b, mode="full")
    corr_norm = corr / corr[(len(corr) // 2)]
    lag = np.arange(-len(serie_a) + 1, len(serie_a), 1)
    return corr_norm, lag

In [ ]:
### ------------------------------------------------------------------------------------
### Define a function for the analysis with vaniable n
### ------------------------------------------------------------------------------------
def cos_series(n):
    ### Create the series
    upper_limit = n * 0.5
    wavelength = 10
    t = np.arange(0, upper_limit, 0.5)
    y = 2 * np.cos((2 * np.pi / wavelength) * t)
    ### Calculate the autocorrelation
    corr_norm, lag = xcorr(y, y)
    ### Make the figure
    fig = plt.figure(figsize=(10, 5), dpi=300)
    ax1 = fig.add_subplot(211)
    ax2 = fig.add_subplot(212)
    title = "Serie (n=" + str(n) + ")"
    ### Subplot 1: Temperature
    ax1.plot(t, y, color="blue")
    ax1.grid(linestyle="dashed", alpha=0.3)
    ax1.set_title(title, loc="left")
    ### Subplot 2: Partial pressure of carbon dioxide in seawater
    ax2.plot(lag, corr_norm, color="black")
    ax2.grid(linestyle="dashed", alpha=0.3)
    ax2.set_title("Autocorrelation", loc="left")
    ax2.set_ylabel("Autocorrelation Coefficient")
    ax2.set_xlabel("Lag Time")
    ### Final touch
    fig.tight_layout(pad=0.2)
    plt.show()

In [ ]:
### ------------------------------------------------------------------------------------
### Result with n=200
### ------------------------------------------------------------------------------------
cos_series(200)

In [ ]:
### ------------------------------------------------------------------------------------
### Result with n=400
### ------------------------------------------------------------------------------------
cos_series(400)

A frequência de amostragem para n=200 é 0.005.

A função xcorr ajusta o coeficiente de autocorrelação de maneira que para o máximo lag time o valor seja 0. Assim, a forma geral se conserva, mas o coeficiente de correlação muda. Por exemplo, para n=200 e um lag time de 100, o valor do coeficiente de autocorrelação é 0.5. Porém, para n=400 e o mesmo lag time de 100, o coeficiente de autocorrelação é maior.

#### 6. Utilize novamente a série da concentração de CO2 de Mauna Loa
**a) Calcule a função de covariância (ou autocorrelação) da série original de CO2 e da série sem tendência calculada anteriormente (questão 4)**


In [ ]:
### ------------------------------------------------------------------------------------
### Make the correlation
### ------------------------------------------------------------------------------------
corr_pco2, lag_pco2 = xcorr(pco2_mau2, pco2_mau2)
corr_dpco2, lag_dpco2 = xcorr(pco2_cub_detre, pco2_cub_detre)

**b) Plote a autocorrelação em função do “lag” para as duas séries (original e detrended). Use a lag (eixo x) variando de 0 a 60 meses**


In [ ]:
### ------------------------------------------------------------------------------------
### Plot the autocorrelation
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
### Subplot 1: Temperature
ax1.plot(lag_pco2, corr_pco2, color="blue")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_xlim(0, 60)
ax1.set_ylim(0.9, 1.01)
ax1.set_title("Autocorrelation monthly mean CO2 mole fraction in Mauna Loa", loc="left")
ax1.set_ylabel("Autocorrelation Coefficient")
ax1.set_xlabel("Lag Time")
### Subplot 2: Partial pressure of carbon dioxide in seawater
ax2.plot(lag_dpco2, corr_dpco2, color="black")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_xlim(0, 60)
ax2.set_title(
    "Autocorrelation detrendend monthly mean CO2 mole fraction in Mauna Loa", loc="left"
)
ax2.set_ylabel("Autocorrelation Coefficient")
ax2.set_xlabel("Lag Time")
### Final touch
fig.tight_layout(pad=0.2)
plt.show()

**c) Descreva o comportamento da autocorrelação para ambas as séries**


No caso da série original, a autocorrelação é uma linha reta com inclinação negativa.  
No caso da série sem tendência, pode ser observado um comportamento periódico, onde os maiores valores de autocorrelação são encontrados de 12 em 12 meses.  
Em resumo, a tendência pode mascarar a verdadeira autocorrelação da série temporal.

**d) Verifique se a série sem tendência linear (detrended) de CO2 obedece à uma distribuição normal. Ou seja, faça o histograma de xd (série detrended) e use a função lillietest.m para verificar se a série possui distribuição normal**


In [ ]:
### ------------------------------------------------------------------------------------
### Plot the histogram
### ------------------------------------------------------------------------------------
plt.figure(dpi=300)
hist = pco2_cub_detre.hist(bins="auto")
hist.set_title(
    "Histogram detrendend monthly mean CO2 mole fraction\nin Mauna Loa", loc="left"
)
hist.set_ylabel("Counts")
plt.grid(linestyle="dashed", alpha=0.3)
plt.show()

In [ ]:
### ------------------------------------------------------------------------------------
### Define a function to test the normal distribution of the data
### ------------------------------------------------------------------------------------
def normal_test(series, alpha):
    # One-sample Kolmogorov-Smirnov test (kst):
    kst = stats.ks_1samp(series, stats.norm.cdf).pvalue
    if kst < alpha:
        print(f"KST (p-values): {round(kst, 2)} < {alpha}")
        print(
            f"Com um nivel de confiança de {100-alpha*100}% o teste de normalidade de Kolmogorov-Smirnov sugere que a série não possui uma distribuição normal"
        )
    else:
        print(f"KST (p-values): {round(kst, 2)} > {alpha}")
        print(
            f"Com um nivel de confiança de {100-alpha*100}% o teste de normalidade de Kolmogorov-Smirnov sugere que a série pode ter uma distribuição normal"
        )
    # Shapiro-Wilk test (swt):
    print("---------------------------------------------------------------------------")
    swt = stats.shapiro(series).pvalue
    if swt < alpha:
        print(f"SWT (p-values): {round(swt, 2)} < {alpha}")
        print(
            f"Com um nivel de confiança de {100-alpha*100}% o teste de normalidade de Shapiro-Wilk sugere que a série não possui uma distribuição normal"
        )
    else:
        print(f"SWT (p-values): {round(swt, 2)} > {alpha}")
        print(
            f"Com um nivel de confiança de {100-alpha*100}% o teste de normalidade de Shapiro-Wilk sugere que a série pode ter uma distribuição normal"
        )
    # D’Agostino and Pearson’s test (dpt):
    print("---------------------------------------------------------------------------")
    dpt = stats.normaltest(series, nan_policy="omit").pvalue
    if dpt < alpha:
        print(f"DPT (p-values): {round(dpt, 2)} < {alpha}")
        print(
            f"Com um nivel de confiança de {100-alpha*100}% o teste de normalidade de D’Agostino and Pearson’s sugere que a série não possui uma distribuição normal"
        )
    else:
        print(f"DPT (p-values): {round(dpt, 2)} > {alpha}")
        print(
            f"Com um nivel de confiança de {100-alpha*100}% o teste de normalidade de D’Agostino and Pearson’s sugere que a série pode ter uma distribuição normal"
        )

In [ ]:
### ------------------------------------------------------------------------------------
### Apply the function
### ------------------------------------------------------------------------------------
normal_test(pco2_cub_detre, 0.05)

**e) Comente os resultados**

A distribuição observada no histograma é semelhante à uma distribuição normal com uma assimetria positiva. Porém, os resultados dos testes estatísticos sugerem que a série não possui uma distribuição normal.

#### 7.	Repita a questão 6, mas agora com a série de SOI. Comente os resultados

In [ ]:
### ------------------------------------------------------------------------------------
### Load the SOI data
### ------------------------------------------------------------------------------------
soi = pd.read_csv("./Data/SOI.csv", skiprows=1)
soi["time"] = pd.to_datetime(soi["Date"], format="%Y%m")
soi = soi.drop(columns=["Date"])
soi = soi.set_index("time").squeeze()

In [ ]:
### ------------------------------------------------------------------------------------
### Detrend the SOI data
### ------------------------------------------------------------------------------------
soi_cub_detre = cubic_detrend(soi)

In [ ]:
### ------------------------------------------------------------------------------------
### Calculate the autocorrelation
### ------------------------------------------------------------------------------------
corr_soi, lag_soi = xcorr(soi, soi)
corr_dsoi, lag_dsoi = xcorr(soi_cub_detre, soi_cub_detre)

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the autocorrelation
### ------------------------------------------------------------------------------------
fig = plt.figure(figsize=(10, 5), dpi=300)
ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
### Subplot 1: Original data
ax1.plot(lag_soi, corr_soi, color="blue")
ax1.grid(linestyle="dashed", alpha=0.3)
ax1.set_xlim(0, 60)
ax1.set_title(
    "Autocorrelation monthly Southern Oscillation Index (SOI) (Original)", loc="left"
)
ax1.set_ylabel("Autocorrelation Coefficient")
ax1.set_xlabel("Lag Time")
### Subplot 2: Detrended data
ax2.plot(lag_dsoi, corr_dsoi, color="black")
ax2.grid(linestyle="dashed", alpha=0.3)
ax2.set_xlim(0, 60)
ax2.set_title(
    "Autocorrelation detrendend monthly Southern Oscillation Index (SOI) (Detrended)",
    loc="left",
)
ax2.set_ylabel("Autocorrelation Coefficient")
ax2.set_xlabel("Lag Time")
### Final touch
fig.tight_layout(pad=0.2)
plt.show()

In [ ]:
### ------------------------------------------------------------------------------------
### Plot the histogram of SOI
### ------------------------------------------------------------------------------------
plt.figure(dpi=300)
hist2 = soi_cub_detre.hist(bins="auto")
hist2.set_title(
    "Histogram detrendend monthly Southern Oscillation Index (SOI)", loc="left"
)
hist2.set_ylabel("Counts")
plt.grid(linestyle="dashed", alpha=0.3)
plt.show()

In [ ]:
### ------------------------------------------------------------------------------------
### Test the normality of the SOI data
### ------------------------------------------------------------------------------------
normal_test(soi_cub_detre, 0.05)

A diferença da série de CO2 atmosférico, a série de SOI não parece ter uma tendência muito marcada, e os valores de autocorrelação são semelhantes usando os dados originais e os dados sem tendência. O histograma é bem semelhante a uma distribuição normal, e um dos testes de normalidade aplicados (Kolmogorov-Smirnov) sugere que a série pode ter uma distribuição normal. Porém, os outros dois testes sugerem que a série não possui uma distribuição normal.